In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import requests
import time
import numpy as np
import os

# List of search queries
queries = ["beach plastic", "beach pollution", "beach litter"]

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Provide the path to your chromedriver executable
driver.get('https://www.google.com/imghp')
time.sleep(2)

# Function to get image URLs for a query
def get_image_urls(query, pages=10):
    search_box = driver.find_element(by=By.NAME, value='q')
    search_box.clear()
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)
    npUrls_filename = f'./npUrls/{query}/urls.npy'
    image_urls = np.array([])
    if os.path.exists(npUrls_filename):
        image_urls = np.load(npUrls_filename)

    for _ in range(pages):  # Scroll and load more images
        ended_msg = driver.find_element(by= By.CSS_SELECTOR, value='.OuJzKb')
        if ended_msg.is_displayed():
            break
        load_btn = driver.find_element(by= By.CSS_SELECTOR, value='.LZ4I')
        if load_btn.is_displayed():
            load_btn.click()
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    images = driver.find_elements(by= By.CSS_SELECTOR, value='.rg_i')
    for index, img in enumerate(images):
        try:
            img.click()
            time.sleep(5)
            full = driver.find_element(by= By.CSS_SELECTOR, value='.iPVvYb')
            img_url = full.get_attribute('src')
            if img_url in image_urls:
                continue
            if img_url and img_url.startswith('http'):
                image_urls = np.concatenate((image_urls, np.array([img_url])))
                npUrls = np.array(image_urls)
                np.save(npUrls_filename, npUrls)
                response = requests.get(img_url)
                if response.status_code == 200:
                    image_extension = img_url.split('.')[-1]
                    image_filename = f'./images/{query}/{index + 1}.{image_extension}'
                    
                    with open(image_filename, 'wb') as f:
                        f.write(response.content)
        except:
            print('failed to load', img.get_attribute('src'))
    
    return image_urls

# Iterate over each query
total_urls = []
for query in queries:
    image_urls = get_image_urls(query)
    print(f"Query: {query}, Number of Images: {len(image_urls)}")
    total_urls.append(image_urls)

# Close the Selenium WebDriver
driver.quit()